In [9]:
with open('datasets/list_attr_celeba.txt', 'r') as f:
    lines = f.readlines()

In [15]:
attrs = lines[1]
attrs = attrs.split(" ")
attrs

['5_o_Clock_Shadow',
 'Arched_Eyebrows',
 'Attractive',
 'Bags_Under_Eyes',
 'Bald',
 'Bangs',
 'Big_Lips',
 'Big_Nose',
 'Black_Hair',
 'Blond_Hair',
 'Blurry',
 'Brown_Hair',
 'Bushy_Eyebrows',
 'Chubby',
 'Double_Chin',
 'Eyeglasses',
 'Goatee',
 'Gray_Hair',
 'Heavy_Makeup',
 'High_Cheekbones',
 'Male',
 'Mouth_Slightly_Open',
 'Mustache',
 'Narrow_Eyes',
 'No_Beard',
 'Oval_Face',
 'Pale_Skin',
 'Pointy_Nose',
 'Receding_Hairline',
 'Rosy_Cheeks',
 'Sideburns',
 'Smiling',
 'Straight_Hair',
 'Wavy_Hair',
 'Wearing_Earrings',
 'Wearing_Hat',
 'Wearing_Lipstick',
 'Wearing_Necklace',
 'Wearing_Necktie',
 'Young',
 '\n']

In [3]:
first_line = first_line.replace(" ", "")

In [4]:
attrs = first_line.split(" ")

In [6]:
attrs

['000001.jpg-111-1-1-1-1-1-1-1-11-1-1-1-1-1-111-11-1-11-1-11-1-1-111-11-11-1-11\n']